# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [54]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [1]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

NameError: name 'nlp' is not defined

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [56]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt', encoding = "utf8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt', encoding = "utf8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [57]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [58]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    
    gold_pred_intersect = len(gold.intersection(pred))
    precision = round(gold_pred_intersect/len(pred), 2)
    print("Precision: "+ str(precision))
    
    recall = round(gold_pred_intersect/len(gold), 2)
    print("Recall: "+ str(recall))
    
    f1 = round(2*(precision * recall)/(precision+recall), 2)
    print("F1: "+ str(f1))

To test your code, you can run the following cell:

In [59]:
evaluation_report(set(range(3)), set(range(5)))

Precision: 0.6
Recall: 1.0
F1: 0.75


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [60]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for row in df.itertuples():
        yield((row.sentence_id, row.beg, row.end))

In [61]:
# Creating generator.
generator = gold_spans(df_dev)

# Counting yielded spans.
print("Number of yielded spans: " + str(sum(1 for x in generator)))

# Checking first and last triple.
n = 1
for triple in gold_spans(df_dev):
    if n in [1, 5917]:
        print(triple)
    n = n+1

Number of yielded spans: 5917
('0946-000', 2, 3)
('1161-010', 1, 3)


To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [62]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [63]:
# Initializing empty set.
spans_dev_pred = set()
# Adding predictions to set by looping over each row in data frame.
for row in range(0, len(df_dev)-1):
    # Extracting sentence_id.
    sentence_id = df_dev.iloc[row, 0]
    # Extracting sentence column and transforming it to nlp-object.
    doc = nlp(df_dev.iloc[row, 1])
    # Looping over each identified entity within sentence.
    for ent in doc.ents:
        # Adding sentence_id and entity position to set.
        spans_dev_pred.add((sentence_id, ent.start, ent.end))

In [64]:
# Evaluating.
evaluation_report(spans_dev_gold, spans_dev_pred)

Precision: 0.52
Recall: 0.7
F1: 0.6


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Write code that prints the false positives and the false negatives from the automatic prediction. Have a look at the output. What are your observations? How could you improve the result? Discuss these questions in a short text.

In [65]:
import numpy as np
'''The implemented function gives an overview about the false positives and false negatives. We can specify the number
   of examples we want to look at. For each example, the wrongly (not) predicted namd entities are printed. 
   By this procedure, it is easy to get an overview about the types of entities which are wrongly (not) predicted.'''

def analyse_error(gold, pred, error_type, n):
# gold:        gold-standard (true) named entities.
# pred:        predicted named entities.
# error_type: "false_positive" or "false_negative".
# n:           number of desired examples printed.
    
    # Extracting either false positives (in prediction set, but not in gold set)
    # or false negatives (not in prediction set, but in gold set).
    if error_type == "false_positive":
        errors = pred - gold
    elif error_type == "false_negative":
        errors = gold - pred
    
    print(str("*************** Identified " + error_type + "s ***************"))
    
    # Initializing counter and empty list to store results in.
    counter = 0
    res = []
    
    # Printing relevant information about false positives / false negatives.
    for each in errors:

        if counter > n:
            break

        # Extracting identified sentence belonging to "each" as list.
        each_sentence = " ".join(df_dev[df_dev["sentence_id"] == each[0]].iloc[:, 1]).split(" ")

        # Extracting (not) predicted entity name considering beginning and end position.
        each_pred_entity_name =  " ".join(np.unique([each_sentence[each[1]], each_sentence[each[2]-1]]))
        res.append(each_pred_entity_name)

        counter = counter + 1
    
    print(res)

In [66]:
# Printing false positives.
analyse_error(spans_dev_gold, spans_dev_pred, "false_positive", n = 50)

*************** Identified false_positives ***************
['71 77', 'Obilic Zeleznik', '83', '65', '70 75', '1', 'midday', '5', '69', '7', '1 September', '4 Peter', '6.', '5', '30 about', '3 9', '1949', '1', '68', '3', 'Third', ', --', '60', '0 1', '0 Torquay', '6.', '1', 'mid-1997', '20 minutes', '3', '$ billion', 'Friday', '6.', '14 May', 'second', '86', '144', 'second', "'s Abu", '10.09', 'first', '70', 'Friday', '0', 'One', '2 Israel', '1/3 7', '69', 'Party the', 'today', 'Thursday']


In [67]:
# Printing false negatives.
analyse_error(spans_dev_gold, spans_dev_pred, "false_negative", n = 50)

*************** Identified false_negatives ***************
['YEREVAN', 'Lad Paddy', 'Final IAAF', 'Agriculture Department', 'Magna', 'Group Islamic', 'Nicol Peter', 'Holy Jerusalem', 'Warwickshire', 'Exchange New', 'Olympic', 'Queens Rangers', 'Moengo', 'King', 'Festival Venice', 'Gugelmin Mauricio', 'Joakim Persson', 'Johnstone St', 'Yastrzhembsky', 'ENGLAND', 'Banisadr', 'Hartlepool', 'Swansea', 'Crewe', 'Healy', 'Cup World', 'EUROBONDS', 'Chen Chun-yeh', 'TORONTO', 'English', 'States United', 'Lara Raul', 'Tranmere', 'Worcestershire', 'C. Tolley', 'LIFFE', 'Nyerere', 'pre-Castro', 'Fal', 'Melrose', 'Gillespie J.', 'Honiball', 'Hampshire', 'Expos', 'Dirk Medved', 'Kaluwitharana', 'Army Irish', 'Kaluwitharana', 'Billie King', 'Coruna Deportivo', 'Warwickshire']


In [68]:
# For the false positives, it can be seen that there are a lot of numbers which are predicted as a named entity. 
# However, these numbers are not labeled by the gold-standard.

# In contrast, the analysis of the false negatives shows us that a lot of persons are not predicted as a named entity. 

# Consequently, we should adjust our predictor so that it does not predict numbers as a named entity. 
# Also, it should make sure that more persons are recognized as named entities.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to pre-process the data and/or post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [69]:
# Since we think that it is much easier to influence the entities that are not predicted by the 
# predictor (numbers) than predicting more persons as entities, 
# we concentrate on the false-positives. We adjust our predictor by excluding predicted entities if they include numbers.

In [70]:
# Initializing empty set.
spans_dev_pred_new = set()
# Adding predictions to set by looping over each row in data frame.
for row in range(0, len(df_dev)-1):
    # Extracting sentence_id.
    sentence_id = df_dev.iloc[row, 0]
    # Extracting sentence column and transforming it to nlp-object.
    doc = nlp(df_dev.iloc[row, 1])
    # Looping over each identified entity within sentence.
    for ent in doc.ents:
        # Removing predicted entity if it includes numbers.
        if ent.label_ != "CARDINAL" and ent.label_ != "DATE" and ent.label_ != "TIME" and ent.label_ != "ORDINAL" and ent.label_ != "QUANTITY":
            # Adding sentence_id and entity position to set.
            spans_dev_pred_new.add((sentence_id, ent.start, ent.end))

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

In [71]:
# Evaluating new predictor.
evaluation_report(spans_dev_gold, spans_dev_pred_new)

Precision: 0.84
Recall: 0.7
F1: 0.76


Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of each span, you can use the special value `--NME--`.

In [72]:
# Initializing empty data frame.
col_names =  ['sentence_id', 'sentence', 'beg', 'end', 'label']
df_dev_new  = pd.DataFrame(columns = col_names)

# Filling data frame.
for pred_named_ent in spans_dev_pred_new:
    sentence_id = pred_named_ent[0]
    sentence = df_dev[df_dev["sentence_id"] == sentence_id].iloc[0, 1]
    beg = pred_named_ent[1]
    end = pred_named_ent[2]
    df_dev_new.loc[len(df_dev_new)] = [sentence_id, sentence, beg, end, "--NME--"]

# Printing head of new data frame.
df_dev_new = df_dev_new.sort_values(by = ["sentence_id"])
df_dev_new.head()

,sentence_id,sentence,beg,end,label
4085,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,0,3,--NME--
3304,0946-001,LONDON 1996-08-30,0,1,--NME--
4088,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
2725,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,--NME--
2698,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [73]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    for row in df.itertuples():
        yield((row.sentence_id, row.beg, row.end, row.label))

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [74]:
def wikipedia_tags(df):
    for each in range(len(df)):
        sentence_id = df.iloc[each, 0]
        sentence = df.iloc[each,1]
        sentence = sentence.split()
        beg = df.iloc[each, 2]
        end = df.iloc[each, 3]
        wikipedia_tag = "_".join(sentence[beg:end])
        yield (sentence_id, beg, end, wikipedia_tag)

## Predictions.
wiki_dev_pred = set(wikipedia_tags(df_dev_new))

## Gold standard.
wiki_dev_gold = set(gold_mentions(df_dev))

# Evaluating.
evaluation_report(wiki_dev_gold, wiki_dev_pred)

Precision: 0.31
Recall: 0.26
F1: 0.28


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on when you click on a link) as mentions of the entities (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [75]:
with bz2.open("kb.tsv.bz2", 'rt', encoding = "utf8") as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [76]:
df_kb.loc[df_kb["mention"] == "Sweden"]

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing stands first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [77]:
def label_as_gold(df):
    for each in range(len(df)):
        sentence = df.iloc[each,1]
        sentence = sentence.split()
        beg = df.iloc[each, 2]
        end = df.iloc[each, 3]
        mention = sentence[beg:end]
        mention = " ".join(mention)
        label = df.iloc[each, 4]
        
        yield (mention, label)

In [78]:
set_dev_gold_label = set(label_as_gold(df_dev))
df_dev_gold_label = label_as_gold(df_dev)
df_dev_gold_label = pd.DataFrame(df_dev_gold_label)
df_dev_gold_label.columns = ['mention', 'label']
df_dev_gold_label.head()

,mention,label
0,LEICESTERSHIRE,Leicestershire_County_Cricket_Club
1,LONDON,London
2,West Indian,West_Indies_cricket_team
3,Phil Simmons,Phil_Simmons
4,Leicestershire,Leicestershire_County_Cricket_Club


In [79]:
def label_most_prob_ent(df):
    for each in range(len(df)):
        mention = df.iloc[each, 0]
        mention = str(mention)
    
        subset = df_kb.loc[df_kb["mention"] == mention]
        subset = subset.sort_values(by=['prob'])
        
        if subset.empty == True:
            label = '--NME--'
        else:
            label = subset.iloc[0,1]
        
        yield (mention, label)
        
set_dev_most_prob_label = set(label_most_prob_ent(df_dev_gold_label))

In [80]:
evaluation_report(set_dev_gold_label, set_dev_most_prob_label)

Precision: 0.74
Recall: 0.7
F1: 0.72


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [81]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as a bag of words containing the mention itself, as well as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [82]:
df_contexts.context[df_contexts.mention == "Lincoln"]

41465    Nebraska Concealed Handgun Permit In Lincoln m...
41466    Lazlo restaurants are located in Lincoln and O...
41467    California Washington Overland Park Kansas Lin...
41468    City Missouri Omaha Nebraska and Lincoln Nebra...
41469    by Sandhills Publishing Company in Lincoln Neb...
                               ...                        
41609                                Lincoln Leyton Orient
41610               English division three Swansea Lincoln
41611    league membership narrowly edging out Lincoln ...
41612                                    Lincoln Cambridge
41613                                              Lincoln
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** To tune the performance of your method, you can try to tweak the behaviour of the vectorizer (for example, should it apply lowercasing or not?) and the width of the window from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [83]:
# First, we implement the dictionary which includes all classifiers for each mention:

# Instantiate pipeline.
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initializing empty dictionary.
mention_classififer_dict = {}

for mention in df_contexts.mention.unique():
    
    # Extracting class_prior.
    class_prior_prob = df_kb[["entity", "prob"]][df_kb["mention"] == mention]
    class_prior_prob = class_prior_prob.sort_values(by = ["entity"]).prob.tolist()
    
    # Setting up classifier pipeline.
    nb_pipeline = make_pipeline(CountVectorizer(lowercase = True), 
                                MultinomialNB(class_prior = class_prior_prob)) 

    # Using pipeline to train naive bayes classifier on training data.
    nb_classifier = nb_pipeline.fit(df_contexts.context[df_contexts.mention == mention], 
                                    df_contexts.entity[df_contexts.mention == mention])
    
    # Adding classifier to dict.
    mention_classififer_dict[mention] = nb_classifier

In [84]:
def label_using_classifier(df):
        
    for row in range(len(df)):

        # Extracting sentence.
        sentence = df.iloc[row, 1].split()

        # Extracting span of mention.
        beg = df.iloc[row, 2]
        end = df.iloc[row, 3]

        # Extracting mention.
        mention = " ".join(sentence[beg:end])

        # Extracting context from sentence.
        # Context consists of mention itself, as well as 5 tokens to the left and 5 tokens to the right of the mention.
        # Converting context to pd.Series format (necessary for classifier).
        indexmin = end-beg
        indexmax = end
        context = sentence[max(0, beg-5) : min(end+5, len(sentence))]
        context = pd.Series(" ".join(context))

        # Predicting label using mention and context.

        ## Checking if entity is in dictionary. If yes, use classifier.
        ## Else, proceed as in problem 5.
       
        if len(df_contexts.entity[df_contexts.mention == mention].unique()) > 1:
            label = mention_classififer_dict[mention].predict(context)[0]
         
        else:
            subset = df_kb.loc[df_kb["mention"] == mention]
            subset = subset.sort_values(by=['prob'])
            if subset.empty == True:
                label = '--NME--'
            else:
                label = subset.iloc[0,1]
            
        yield (mention, label)

set_dev_classified_label = set(label_using_classifier(df_dev))


In [85]:
evaluation_report(set_dev_gold_label, set_dev_classified_label)

Precision: 0.8
Recall: 0.78
F1: 0.79


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>